<a target="_blank" href="https://colab.research.google.com/github/UpstageAI/cookbook/blob/main/Solar-Fullstack-LLM-101/14_Reasoning.ipynb">
<img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# Auto Marketing 
Using product descriptions and corresponding webpage content, Solar will create marketing email messages automatically.

In [1]:
! pip3 install -qU langchain langchain-upstage langchain_community python-dotenv firecrawl-py==0.0.20 marko

## UPSTAGE_API_KEY
To obtain your Upstage API key, follow these steps:

1. Visit the Upstage AI console at <https://console.upstage.ai>.
2. Sign up for an account if you don't already have one.
3. Log in to your account.
4. Navigate to the API key section.
5. Generate your API key.
6. Copy the key and save it securely.

![Console](./figures/console.upstage.ai.jpg)

In [2]:
# @title set API key
import os
import getpass

from pprint import pprint

import warnings

warnings.filterwarnings("ignore")

if "google.colab" in str(get_ipython()):
    # Running in Google Colab. Please set the UPSTAGE_API_KEY in the Colab Secrets
    from google.colab import userdata

    os.environ["UPSTAGE_API_KEY"] = userdata.get("UPSTAGE_API_KEY")
else:
    # Running locally. Please set the UPSTAGE_API_KEY in the .env file
    from dotenv import load_dotenv

    load_dotenv()

if "UPSTAGE_API_KEY" not in os.environ:
    os.environ["UPSTAGE_API_KEY"] = getpass.getpass("Enter your Upstage API key: ")

In [3]:
# We will use FireCrawl, https://python.langchain.com/docs/integrations/document_loaders/firecrawl/
# You will need to set FIRECRAWL_API_KEY. Learn mode https://firecrawl.dev/.


if "FIRECRAWL_API_KEY" not in os.environ:
    os.environ["FIRECRAWL_API_KEY"] = getpass.getpass("Enter your Firecrawl API key: ")

In [4]:
product_info = """
UpstageAI provides AI-driven tools and solutions aimed at enhancing productivity, automating tasks, and assisting decision-making for businesses. Their core offerings include:

1. **Solar Pro Preview**: A cutting-edge, highly intelligent large language model (LLM) that runs efficiently on a single GPU, designed to support complex tasks like data analysis and decision support. It's particularly suitable for businesses seeking advanced AI capabilities.
   
2. **Solar Mini**: A purpose-trained LLM tailored for specific tasks, providing businesses with powerful yet efficient AI functionalities.
   
3. **Document AI Tools**: These include Document Parsing, OCR (Optical Character Recognition), and Key Information Extraction, all designed to streamline document handling. Users can extract tables, figures, and text while enabling automation workflows for data management.

4. **Task APIs**: Upstage offers several APIs, such as:
   - **Chat**: For building conversational agents.
   - **Translation**: Context-aware English-Korean translation.
   - **Embeddings**: For text-to-vector conversion.
   - **Groundedness Check**: Ensures AI-generated responses are based on accurate data.
   - **Text-to-SQL**: Converts natural language into SQL queries (coming soon).

5. **Industry-Specific Intelligence**: Upstage is developing specialized APIs for industries such as healthcare, finance, and law, providing domain-specific AI solutions (coming soon).

Upstage focuses on automating repetitive tasks, improving business decision-making, and providing generative business intelligence tools to increase efficiency across various sectors.
"""

In [5]:
from langchain_community.document_loaders import FireCrawlLoader

target_page = "https://www.firecrawl.dev/"
loader = FireCrawlLoader(url=target_page, mode="scrape")
docs = loader.load()

In [6]:
pprint(docs[0].metadata)

{'description': 'Firecrawl crawls and converts any website into clean '
                'markdown.',
 'keywords': 'Firecrawl,Markdown,Data,Mendable,Langchain',
 'language': 'en',
 'ogDescription': 'Turn any website into LLM-ready data.',
 'ogImage': 'https://www.firecrawl.dev/og.png?123',
 'ogLocaleAlternate': [],
 'ogSiteName': 'Firecrawl',
 'ogTitle': 'Firecrawl',
 'ogUrl': 'https://www.firecrawl.dev/',
 'pageStatusCode': 200,
 'robots': 'follow, index',
 'sitemap': {'changefreq': 'weekly', 'lastmod': '2024-09-16T18:04:06.184Z'},
 'sourceURL': 'https://www.firecrawl.dev/',
 'title': 'Home - Firecrawl'}


In [7]:
import marko
from marko.inline import RawText, Emphasis, CodeSpan, Link, Image
import re


def extract_clean_text(markdown_text):
    # Parse the markdown into an AST
    ast = marko.parse(markdown_text)

    clean_text = []

    def traverse(node):
        # If the node is a RawText, append its content
        if isinstance(node, RawText):
            clean_text.append(node.children)
        # If the node has children, recurse into them unless it's a link or image
        elif hasattr(node, "children") and not isinstance(node, (Link, Image)):
            for child in node.children:
                traverse(child)
        # For other node types that contain text (e.g., Emphasis, CodeSpan), handle accordingly
        elif isinstance(node, (Emphasis, CodeSpan)):
            traverse(node.children if hasattr(node, "children") else node)
        # You can add more conditions here if you want to handle other specific node types

    # Start traversing from the root of the AST
    traverse(ast)

    # Join all extracted text parts with spaces and clean up whitespace
    text = " ".join(clean_text).strip()

    # Remove URLs
    text = re.sub(r"http[s]?://\S+", "", text)
    text = re.sub(r"\bwww\.\S+", "", text)

    return text


# Use the function to extract clean text
target_info = extract_clean_text(docs[0].page_content)

# Print the cleaned text
pprint(target_info[:100])

('Our first Launch Week is over!  Beta Features     13.9k Turn websites '
 'into    LLM-ready  data Power ')


In [8]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_upstage import ChatUpstage

marketing_prompt_teample = ChatPromptTemplate.from_messages(
    [
        (
            "human",
            """You are a marketing specialist at UpsatgeAI selling the product.
Write a convincing and professional email message to a target company to sell your product.
Use the provided product information and details about the target company to personalize the message effectively.
Make your message relevant to the target company.

---
**Requirements:**
- The email should have a clear subject line.
- Highlight how your product can solve specific problems or add value to the target company.
- Maintain a professional and engaging tone.
- Include a call to action encouraging the recipient to take the next step.

---  
**Target Company Information to Email:** 
{target_company_info}
---   
**Your Product to Sell:** 
{product_info}

""",
        )
    ]
)

In [9]:
# write prompt
prompt = marketing_prompt_teample.format_messages(
    target_company_info=target_info,
    product_info=product_info,
)

print(prompt[0].content)

You are a marketing specialist at UpsatgeAI selling the product.
Write a convincing and professional email message to a target company to sell your product.
Use the provided product information and details about the target company to personalize the message effectively.
Make your message relevant to the target company.

---
**Requirements:**
- The email should have a clear subject line.
- Highlight how your product can solve specific problems or add value to the target company.
- Maintain a professional and engaging tone.
- Include a call to action encouraging the recipient to take the next step.

---  
**Target Company Information to Email:** 
Our first Launch Week is over!  Beta Features     13.9k Turn websites into    LLM-ready  data Power your AI apps with clean data crawled from any website. It's also open-source. Start for free (500 credits)Start for free  A product by Crawl, Scrape, Clean We crawl all accessible subpages and give you clean markdown for each. No sitemap required.

In [10]:
llm = ChatUpstage(model="solar-1-mini-chat")
chain = marketing_prompt_teample | llm | StrOutputParser()

email_msg = chain.invoke(
    {
        "target_company_info": target_info,
        "product_info": product_info,
    }
)

print(email_msg)

Subject: Enhance Your Business Intelligence with UpstageAI's AI-Driven Solutions

Dear [Target Company],

I hope this email finds you well. I am reaching out to introduce you to UpstageAI, a leading provider of AI-driven tools and solutions designed to enhance productivity, automate tasks, and support decision-making for businesses.

As a company that specializes in turning websites into clean, LLM-ready markdown, we believe that our suite of AI solutions can complement your existing offerings and help you provide even more value to your clients.

Our core products include:

1. **Solar Pro Preview**: A highly intelligent, single GPU-efficient large language model (LLM) suitable for complex tasks like data analysis and decision support.
2. **Solar Mini**: A purpose-trained LLM tailored for specific tasks, providing powerful yet efficient AI functionalities.
3. **Document AI Tools**: These include Document Parsing, OCR (Optical Character Recognition), and Key Information Extraction, stre

In [11]:
llm = ChatUpstage(model="solar-pro")
chain = marketing_prompt_teample | llm | StrOutputParser()

email_msg = chain.invoke(
    {
        "target_company_info": target_info,
        "product_info": product_info,
    }
)

print(email_msg)

Subject: Boost Your Business Productivity with UpstageAI's AI-Powered Tools

---

**Unlock Efficiency and Innovation with UpstageAI's AI Solutions**

Dear [Target Company],

As a marketing specialist at UpsageAI, I'm excited to introduce you to our cutting-edge AI-driven tools and solutions designed to streamline your operations, automate repetitive tasks, and enhance decision-making. Our product suite, tailored for businesses like yours, includes:

1. **Solar Pro Preview:** A highly intelligent large language model (LLM) that runs efficiently on a single GPU, ideal for complex tasks like data analysis and decision support.

2. **Solar Mini:** A purpose-trained LLM for specific tasks, providing powerful yet efficient AI functionalities.

3. **Document AI Tools:** Streamline document handling with Document Parsing, OCR, and Key Information Extraction, enabling automation workflows for efficient data management.

4. **Task APIs:** Leverage our APIs for conversational agents (Chat), conte